In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
results = np.array(range(1,7))
def roll_dice(n, good=True):
    good_dice = np.array([1/6]*6)
    bad_dice = np.array([1/10]*5 + [1/2])
    p1 = np.float128(0.04)
    p2 = np.float128(0.05)
    res = []
    colors = []
    while n > 0:
        dice = good_dice if good else bad_dice
        res.append(np.random.choice(results, size=1, p=dice)[0])
        colors.append(0 if good else 1)
        n -= 1
#         print(good)
        if good and np.random.rand() <= p1:
            good = False
            continue
        if (not good) and np.random.rand() <= p2:
            good = True
    return np.array(res), np.array(colors)

In [8]:
# function by Michał Zobniów
def plot_seq(roll_seq, color):
    if len(roll_seq) > 10000:
        l = 5000
    else:
        l = len(roll_seq)
    colors = ['g', 'r', 'black']
    plt.figure(figsize=(40,5))
    timeTM = np.linspace(0, l/2, l)
    for i, x in enumerate(timeTM): 
        plt.plot([x, x], [0, roll_seq[i]], c=colors[color[i]], )
    plt.xlim(0, 500)
    plt.ylim(0,6)
    plt.show()

In [9]:
def a(i, j):
    if j == 0:
        if i == 1:
            return 1
        else:
            return 0
    s = [p1, p2]
    if i == 1:
        if j == 1:
            return 1-s[0]
        else:
            return s[0]
    else:
        if j == 2:
            return 1-s[1]
        else:
            return s[1]

        
def b(i,j,k):
    dice = good_dice if j == 1 else bad_dice
    return dice[k-1]

def alpha(o):
    res = [[a(1,0)], [a(2,0)]]
    for t in range(1, len(o)+1):
        res[0].append(sum(res[i-1][-1]*a(i,1)*b(i,1,o[t-1])*5 for i in range(1,3)))
        res[1].append(sum(res[i-1][-1]*a(i,2)*b(i,2,o[t-1])*5 for i in range(1,3)))
    return np.array(res)
    
    
def beta(o):
    res = [[1],[1]]
    for t in range(len(o), 0, -1):
        res[0].insert(0, sum(a(1,j) * b(1,j,o[t-1]) * res[j-1][0] *5 for j in range(1,3)))
        res[1].insert(0, sum(a(2,j) * b(2,j,o[t-1]) * res[j-1][0] *5 for j in range(1,3)))
    return np.array(res)

In [13]:
class ksi:
    def __init__(self,o):
        self.o = o
        self.b_caret = self.b_caret_make(o)
        
    def b_caret_make(self,o):
        gamma = alpha(o) * beta(o)
        gamma = gamma
        res = [[], []]
        for k in results:
            ot = np.array([e for e in o if e == k])
            gammat = alpha(ot) * beta(ot)
            gammat = gammat
            res[0].append(np.sum(gammat[0])/np.sum(gamma[0]))
            res[1].append(np.sum(gammat[1])/np.sum(gamma[1]))
        return np.array(res)

    

    def prawie_ksi(self,t, o, first=True, prev_ksi = np.array([])):
        if first:
            res = []
            for i in range(1,3):
                inres = []
                for j in range(1,3):
                    inres.append(alpha(o)[i][t-1] * beta(j)[t] * a(i,j) * b(i,j,o[t]))
                res.append(inres)
            return np.array(res)
        res = []
        for i in range(1,3):
            inres = []
            for j in range(1,3):
                inres.append(alpha(o)[i][t-1] * beta(j)[t] * self.a_caret(i,j, prev_ksi) * self.b_caret[i-1][o[t]])
            res.append(inres)
        return np.array(res)


    def find_ksi(self,o, end):
        old_ksi = []
        for t in range(1,len(o)+1):
            old_ksi.append(self.prawie_ksi(t, o))
        for _ in range(end):
            new_ksi = []
            for t in range(1,len(o)+1):
                new_ksi.append(self.prawie_ksi(t, o, False, np.array(old_ksi)))
            old_ksi = new_ksi
        return new_ksi

    def alpha_caret(self,o, i, j, ksi):
        np.sum(ksi[t][i-1][j-1] for t in range(len(o)))/np.sum(ksi[t][i-1][k] for t in range(len(o)) for k in range(2))

In [14]:
roll = roll_dice(1000)
ks = ksi(roll)

NameError: name 'p1' is not defined